# Voltammetry Simulations
From Compton *et al.* "Understanding voltammetry: simulation of electrode processes", 2014

## Cyclic Voltammogram (reversible)

In [29]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib widget
def cv_rev(sigma):

    #Specify simulation parameters
    theta_i = 10.0
    theta_v = -10.0
    deltaX = 2e-3
    deltaTheta = 0.02

    #Calculate other parameters
    deltaT = deltaTheta / sigma
    maxT = 2 * np.abs(theta_v - theta_i) / sigma
    maxX = 6*np.sqrt(maxT)
    n = int( maxX / deltaX ) # number of spacesteps
    m = int( maxT / deltaT ) # number of timesteps

    # Calculate Thomas coefficients
    wambda = deltaT / (deltaX**2)
    alpha = -wambda
    beta = 2.0*wambda + 1.0
    gamma = -wambda

    # Create containers
    g_mod = np.zeros(n)
    C = np.zeros(n)# concentration profile
    Thetas = np.zeros(m)
    fluxes = np.zeros(m)

    #Modify gamma coefficients 
    g_mod[0] = 0 # boundary condition
    for i in range(1,n):
        g_mod[i] = gamma / (beta - g_mod[i-1] * alpha)
        i+=1

    # BEGIN SIMULATION
    Theta = theta_i

    for k in tqdm(range(m*2)):
        if( k%m < m / 2 ):
            Theta -= deltaTheta
        else:
            Theta += deltaTheta

        # Forward sweep - create modified deltas
        C[0] = (1.0 / (1.0 + np.exp(-Theta)))

        for i in range(1,n-1):
            C[i] = (( C[i] - C[i-1] * alpha ) / ( beta - g_mod[i-1] * alpha ))
            i+=1

        # Back Substitution
        C[n-1] = 1.0

        for i in np.arange(n-2,-1,-1):

            C[i] = C[i] - g_mod[i] * C[i+1]
            i-=1

        #Output current
        flux = -(-C[2] + 4*C[1] -3*C[0]) / (2*deltaX) 

        if(k>=m):
            fluxes[k%m] = flux
            Thetas[k%m] = Theta

        k+=1

    return Thetas, fluxes
    # END SIMULATION
Thetas, Fluxes = cv_rev(100)
plt.plot(Thetas, Fluxes)
Thetas, Fluxes = cv_rev(1000)
plt.plot(Thetas, Fluxes)
Thetas, Fluxes = cv_rev(10000)
plt.plot(Thetas, Fluxes)

100%|██████████| 4000/4000 [00:10<00:00, 372.82it/s]


FigureCanvasNbAgg()

100%|██████████| 4000/4000 [00:01<00:00, 3774.15it/s]


## Cyclic Voltammogram (irreversible)

In [28]:
def cv_irrev(K_0):


    
    #Specify simulation parameters
    theta_i = 10.0
    theta_v = -10.0
    sigma = 10e3
    deltaX = 2e-3
    deltaTheta = 0.02
    alpha_BV = 0.5
    
    C_Abulk = 0.5
    C_Bbulk = 1 - C_Abulk
    
    h = deltaX
    
    def f_BV(Theta):
        return np.exp(-alpha_BV*Theta)
    
    #Calculate other parameters
    deltaT = deltaTheta / sigma
    maxT = 2 * np.abs(theta_v - theta_i) / sigma
    maxX = 6*np.sqrt(maxT)
    n = int( maxX / deltaX ) # number of spacesteps
    m = int( maxT / deltaT ) # number of timesteps

    # Calculate Thomas coefficients
    alpha = 0
    beta = 1 + h*f_BV(theta_i)*K_0*(1+np.exp(theta_i))
    gamma = -1
    delta = h*f_BV(theta_i)*K_0*np.exp(theta_i)

    # Create containers
    b_mod = np.zeros(n)
    d_mod = np.zeros(n)
    g_mod = np.zeros(n)

    C_A = np.zeros(n)# concentration profile of A
    C_B = np.zeros(n)# concentration profile of A    
    Thetas = np.zeros(m)
    fluxes = np.zeros(m)

    #Modify beta, delta coefficients 
    b_mod[0] = beta # boundary condition?
    d_mod[0] = delta # boundary condition?



    # BEGIN SIMULATION
    Theta = theta_i

    for k in tqdm(range(m*2)) :
        if( k%m < m / 2 ):
            Theta -= deltaTheta
        else:
            Theta += deltaTheta
        
        g_mod[0] = 0 # boundary condition
   
        for i in range(1,n):
            g_mod[i] = gamma / (beta - g_mod[i-1] * alpha)
            i+=1

        # Forward sweep - create modified deltas
        C_A[0] = (1.0 / (1.0 + np.exp(-Theta)))

        for i in range(1,n-1):
            beta = 1 + h*f_BV(Theta)*K_0*(1+np.exp(Theta))
            delta = h*f_BV(Theta)*K_0*np.exp(Theta)
            C_A[i] = C_A[i-1] * beta - delta * C_Bbulk
            C_B[i] = 1 + C_Bbulk - C_A[i]
            i+=1

        # Back Substitution
        C_A[n-1] = C_Abulk
        C_B[n-1] = 1 + C_Bbulk - C_A[n-1]

        for i in np.arange(n-2,-1,-1):

            C_A[i] = C_A[i] - g_mod[i] * C_A[i+1]
            i-=1

        #Output current
        flux = (C_A[1] - C_A[0]) / h 

        if(k>=m):
            fluxes[k%m] = flux
            Thetas[k%m] = Theta

        k+=1

    return Thetas, fluxes
    # END SIMULATION
    
# Thetas, Fluxes = sim(10)
# plt.plot(Thetas, Fluxes)
# Thetas, Fluxes = sim(100)
# plt.plot(Thetas, Fluxes)
Thetas, Fluxes = cv_irrev(.1)
plt.plot(Thetas, Fluxes)

NameError: name 'sim' is not defined